<a href="https://colab.research.google.com/github/hegxiten/ECE561MachineVision/blob/master/HW5_Q3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Use the testing mode for evaluation (the whole ready-to-go model), since we 
# are not contributing to the model itself
# If user would like to use this acceleration, select the menu option 
# "Runtime" -> "Change runtime type", select "Hardware Accelerator" -> "GPU" and
# click "SAVE"

%matplotlib inline
import torch,os,torchvision
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import DataLoader, Dataset
from torchvision.models.resnet import ResNet, BasicBlock
from torchvision import datasets, models, transforms
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, CenterCrop, Grayscale

from tqdm.autonotebook import tqdm
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.metrics import confusion_matrix, precision_score, recall_score, f1_score, accuracy_score
import inspect
import time
from PIL import Image

C:\Users\wangz\.conda\envs\py36cv\lib\site-packages\tqdm\autonotebook.py:17: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  " (e.g. in jupyter console)", TqdmExperimentalWarning)


In [3]:
# # Colab library to upload files to notebook
# from google.colab import files

# # Install Kaggle library
# !pip install -q kaggle
# Upload kaggle API key file
# uploaded = files.upload()
# moved 

In [0]:
# Upload kaggle API key file
uploaded = files.upload()
# moved 

Saving kaggle.json to kaggle (1).json
'kaggle (1).json'   kaggle.json   sample_data/


In [12]:
# Mount Google Drive to access pictures
from google.colab import drive
drive.mount('/content/gdrive')
# Check if Google Drive is mounted
%cd /content/gdrive
%ls
%cd /content/gdrive/My\ Drive/Colab\ Notebooks/HW5_files/dogs-vs-cats/

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).
/content/gdrive
'My Drive'/
/content/gdrive/My Drive/Colab Notebooks/HW5_files/dogs-vs-cats


In [0]:
# Download data for the nyc_taxi_trip_duration challenge
!kaggle competitions download -c dogs-vs-cats

  0% 0.00/86.8k [00:00<?, ?B/s]
100% 86.8k/86.8k [00:00<00:00, 34.0MB/s]
 97% 263M/271M [00:09<00:00, 48.1MB/s]
100% 271M/271M [00:10<00:00, 27.8MB/s]
 96% 521M/543M [00:15<00:01, 22.3MB/s]
100% 543M/543M [00:15<00:00, 36.1MB/s]


In [4]:
# Fetch 10 different customized images from mounted Google Drive, 
root_path = '/content/gdrive/My Drive/Colab Notebooks/HW5_files/dogs-vs-cats/'
root_path = 'C:\\Users\\wangz\\Documents\\GitHub\\ECE561MachineVision\\Homework\\hw5\\dogs-vs-cats'
traindir = root_path + '\\train'
testdir = root_path + '\\test1'

trainfiles = os.listdir(traindir)
testfiles = os.listdir(testdir)


In [5]:
class CatDogDataset(Dataset):
    def __init__(self, filelist, root_dir, transform=None):
        self.filelist = filelist
        self.root_dir = root_dir
        self.transform = transform
    
    def __len__(self):
        return len(self.filelist)
    
    def __getitem__(self, idx):
        img_name = self.filelist[idx]
        fullname = os.path.join(self.root_dir, img_name)
        image = Image.open(fullname)
        if "cat" in img_name:
            cls = 0
        elif "dog" in img_name:
            cls = 1
        else:
            raise Exception("Input Image Not Labelled Correctly")
        if self.transform:
            image = self.transform(image)
        return [image, cls]

def get_data_loaders(train_batch_size=256, val_batch_size=64):    
    # Ugly method to hardcode convert 1 channel grayscale to 3 channel RGB-like
    # but it is still gray input (due to MNIST from torchvision) 
    # See https://discuss.pytorch.org/t/how-to-get-mnist-data-from-torchvision-with-three-channels-for-some-pretrained-model-like-vgg/21872
    data_transform = Compose([Resize((224,224)),
                              CenterCrop(224),
                              ToTensor(), 
                              Normalize(mean=[0.485, 0.456, 0.406],
                                        std=[0.229, 0.224, 0.225])])

    train_loader = DataLoader(CatDogDataset(trainfiles, 
                                            traindir, 
                                            transform=data_transform),
                              batch_size=train_batch_size, shuffle=True)

    val_loader = DataLoader(CatDogDataset(trainfiles, 
                                          traindir, 
                                          transform=data_transform),
                            batch_size=val_batch_size, shuffle=False)
    # use the train dataset for both train and validation (no labels on testset)
    return train_loader, val_loader

def calculate_metric(metric_fn, true_y, pred_y):
    # multi class problems need to have averaging method
    if "average" in inspect.getfullargspec(metric_fn).args:
        return metric_fn(true_y, pred_y, average="macro")
    else:
        return metric_fn(true_y, pred_y)
    
def print_scores(p, r, f1, a, batch_size):
    for name, scores in zip(("precision", "recall", "F1", "accuracy"), (p, r, f1, a)):
        print(f"\t{name.rjust(14, ' ')}: {sum(scores)/batch_size:.4f}")

In [6]:
model_1 = torch.hub.load('pytorch/vision', 'resnet18', pretrained=True)

NUM_CLASS = 2
in_fc_nums = model_1.fc.in_features #resnet18 number of the last input neurons
fc = nn.Linear(in_fc_nums,  NUM_CLASS)
model_1.fc = fc


epochs = 5
train_loader, val_loader = get_data_loaders(64, 16)# put your data loader here
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, try Adam this time to have a taste
# optimize the full connection layer only, magic numbers from online resource
optimizer = optim.Adam(model_1.fc.parameters(), lr=0.001, betas=(0.9, 0.999))

os.environ['CUDA_VISIBLE_DEVICES']='0'

start_ts = time.time()
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True
if use_cuda and torch.cuda.is_available():
    torch.cuda.empty_cache()
    model_1.cuda()

losses = []

batches = len(train_loader)
val_batches = len(val_loader)

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    model_1.train()
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        # training step for single batch
        model_1.zero_grad()
        outputs = model_1(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    true_cls, pred_cls = [], []
    # set model to evaluating (testing)
    model_1.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            # Get's the prediction (outputs) from the network
            outputs = model_1(X) 

            val_losses += loss_function(outputs, y)

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            true_cls.extend(y.cpu())
            pred_cls.extend(predicted_classes.cpu())
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(calculate_metric(metric, 
                                            y.cpu(), predicted_classes.cpu()))
        print(confusion_matrix(true_cls, pred_cls, labels=[0,1]))
        print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
        print_scores(precision, recall, f1, accuracy, val_batches)
        losses.append(total_loss/batches) # for plotting learning curve
print(f"Training time: {time.time()-start_ts}s")

Using cache found in C:\Users\wangz/.cache\torch\hub\pytorch_vision_master


C:\Users\wangz\.conda\envs\py36cv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)
C:\Users\wangz\.conda\envs\py36cv\lib\site-packages\sklearn\metrics\classification.py:1439: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no true samples.
  'recall', 'true', average, warn_for)


[[12320   180]
 [  376 12124]]
Epoch 1/5, training loss: 0.1190440043988054, validation loss: 0.06334646046161652
	     precision: 0.8528
	        recall: 0.8417
	            F1: 0.8470
	      accuracy: 0.9777


KeyboardInterrupt: 

In [ ]:
model_2 = torch.hub.load('pytorch/vision', 'resnet34', pretrained=True)
NUM_CLASS = 2
in_fc_nums = model_2.fc.in_features #resnet18 number of the last input neurons
fc = nn.Linear(in_fc_nums,  NUM_CLASS)
model_2.fc = fc

epochs = 5
train_loader, val_loader = get_data_loaders(16,4)# put your data loader here
loss_function = nn.CrossEntropyLoss() # your loss function, cross entropy works well for multi-class problems

# optimizer, try Adam this time to have a tast
# optimize the full connection layer only, magic numbers from online resource
optimizer = optim.Adam(model_2.fc.parameters(), lr=0.001, betas=(0.9, 0.999))

start_ts = time.time()
os.environ['CUDA_VISIBLE_DEVICES']='0'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
use_cuda = True
if use_cuda and torch.cuda.is_available():
    torch.cuda.empty_cache()
    model_2.cuda()

losses = []

losses = []

batches = len(train_loader)
val_batches = len(val_loader)

# loop for every epoch (training + evaluation)
for epoch in range(epochs):
    total_loss = 0

    # progress bar
    progress = tqdm(enumerate(train_loader), desc="Loss: ", total=batches)

    # ----------------- TRAINING  -------------------- 
    # set model to training
    model_2.train()
    for i, data in progress:
        X, y = data[0].to(device), data[1].to(device)
        
        # training step for single batch
        model_2.zero_grad()
        outputs = model_2(X)
        loss = loss_function(outputs, y)
        loss.backward()
        optimizer.step()

        # getting training quality data
        current_loss = loss.item()
        total_loss += current_loss

        # updating progress bar
        progress.set_description("Loss: {:.4f}".format(total_loss/(i+1)))
        
    # releasing unceseccary memory in GPU
    if torch.cuda.is_available():
        torch.cuda.empty_cache()
    
    # ----------------- VALIDATION  ----------------- 
    val_losses = 0
    precision, recall, f1, accuracy = [], [], [], []
    
    # set model to evaluating (testing)
    model_2.eval()
    with torch.no_grad():
        for i, data in enumerate(val_loader):
            X, y = data[0].to(device), data[1].to(device)
            # Get's the prediction (outputs) from the network
            outputs = model_2(X) 

            predicted_classes = torch.max(outputs, 1)[1] # get class from network's prediction
            true_cls.extend(y.cpu())
            pred_cls.extend(predicted_classes.cpu())
            
            # calculate P/R/F1/A metrics for batch
            for acc, metric in zip((precision, recall, f1, accuracy), 
                                   (precision_score, recall_score, f1_score, accuracy_score)):
                acc.append(calculate_metric(metric, 
                                            y.cpu(), predicted_classes.cpu()))
        print(confusion_matrix(true_cls, pred_cls, labels=[0,1]))
        print(f"Epoch {epoch+1}/{epochs}, training loss: {total_loss/batches}, validation loss: {val_losses/val_batches}")
        print_scores(precision, recall, f1, accuracy, val_batches)
        losses.append(total_loss/batches) # for plotting learning curve
print(f"Training time: {time.time()-start_ts}s")